In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

open_ai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")


In [2]:
import json

folder_path="data"
games = []

for file in os.listdir(folder_path):
    if file.endswith(".json"):
        with open(os.path.join(folder_path, file)) as f:
            data = json.load(f)
            games.append(data)
print("Games Loaded:", len(games))

Games Loaded: 5


In [3]:
documents = []
metadatas = []
ids = []
game_titles = [g["title"].lower() for g in games]

for i, game in enumerate(games):
    platforms = game.get('platforms') or []
    if not isinstance(platforms,list):
        platforms = [str(platforms)]
        
    text = f"""
    Title: {game.get('title')}
    Genre: {game.get('genre')}
    Release Date: {game.get('release_date')}
    Platforms: {', '.join(platforms)}
    Publisher: {game.get('publisher')}
    Description: {game.get('description')}
    """
    documents.append(text)
    metadatas.append({"title": game.get("title")})
    ids.append(str(i))
print("Documents created:", len(documents))

Documents created: 5


In [4]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient(path="./games_db")

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

collection = client.get_or_create_collection(
    name="games",
    embedding_function=embedding_function
)



/Users/shannon.adams/Library/CloudStorage/OneDrive-Accenture/repos/Udaplay_01_solution_project.ipynb/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2404.74it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [5]:
collection.add(documents=documents, metadatas=metadatas, ids=ids)

 

In [6]:
def retrieve_game(query):
    results = collection.query(
        query_texts=[query],
        n_results=3
    )
 
    
    documents = results["documents"][0]
    distances = results["distances"][0]
    return documents,distances

In [7]:
def evaluate_retrieval(distances):
    best_score = distances[0]
    
    if best_score < 0.65:
        return "HIGH"
    else:
        return "LOW"

In [8]:
 
 
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=tavily_key)

def game_web_search(query):
    response = tavily_client.search(query=query)
    return response["results"]

In [9]:
def answer_question(query):
    documents, distances = retrieve_game(query)
    confidence = evaluate_retrieval(distances)
    
    if confidence == "HIGH" and documents:
        return {
            "answer": documents[0],
            "confidence": confidence,
            "source": "Local DB"
        }
        
    web_results = game_web_search(query)
    if web_results:
        return {
        "answer": web_results[0]["content"],
        "confidence": "LOW",
        "source": "Web Search (Tavily)"
        }
    return {
        "answer": "No information found"
    }

In [10]:
queries = [
    "When was It Takes Two released?",
    "Who publishes League of Legends?",
    "What types of WoW characters are there?",
    "How to play Raft?",
    "What plaftorms can you play Ravenswatch on?"
]

for q in queries: 
    print("\nQUESTION:", q)
    result = answer_question(q)
    print(result["answer"])
    print("Confidence:", result["confidence"])
    print("Source:", result["source"])


QUESTION: When was It Takes Two released?

    Title: It Takes Two
    Genre: Action-Adventure Platformer
    Release Date: 2021-03-26
    Platforms: PC, PlayStation, Xbox, Nintendo Switch
    Publisher: Electronic Arts
    Description: A cooperative action-adventure platformer where two players work together to navigate imaginative levels and repair a fractured relationship through creative gameplay mechanics.
    
Confidence: HIGH
Source: Local DB

QUESTION: Who publishes League of Legends?

    Title: League of Legends
    Genre: MOBA
    Release Date: 2009-10-27
    Platforms: PC
    Publisher: Riot Games
    Description: A competitive multiplayer online battle arena game where teams of champions battle to destroy the opposing team's Nexus using strategic teamwork, abilities, and map control.
    
Confidence: HIGH
Source: Local DB

QUESTION: What types of WoW characters are there?
They used to use melee weapons and ranger. Although he was a World Shaman, a spec of its own but now 